In [1]:
import requests
import pandas as pd
import domojupyter as domo
url_token = "https://accounts.snapchat.com/login/oauth2/access_token"
payload_token = (
    "grant_type=refresh_token&refresh_token=XXXXXXXX&client_secret=XXXXXXXX"
)
headers_token = {
    'Content-Type': 'application/x-www-form-urlencoded',
    'Cookie': 'XXXXXXXXXXX'
}

response_token = requests.post(url_token, headers=headers_token, data=payload_token)

if response_token.status_code == 200:

    access_token = response_token.json().get("access_token", "")

    urls = {
        "Main": "https://api.snapkit.com/v1/stories/studio/revenue/creator/XXXXXXX/stories?start_date=2024-01-01&end_date=2026-01-01",
        "Spanish": "https://api.snapkit.com/v1/stories/studio/revenue/creator/XXXXXXX/stories?start_date=2024-01-01&end_date=2026-01-01",
        "French": "https://api.snapkit.com/v1/stories/studio/revenue/creator/XXXXXX/stories?start_date=2024-01-01&end_date=2026-01-01",
        "Arabic": "https://api.snapkit.com/v1/stories/studio/revenue/creator/XXXXXX/stories?start_date=2024-01-01&end_date=2026-01-01"
    }
    
    headers_data = {
        'accept': 'application/json',
        'authorization': f'Bearer {access_token}'
    }
    
    df_list = []

    for name, url in urls.items():
        response = requests.get(url, headers=headers_data)
        if response.status_code == 200:
            data = response.json()
            
            if 'stories' in data:
                stories_data = data['stories']
                
                df_stories = pd.json_normalize(stories_data)
                
                df_stories['Snap.Channel'] = name
           
                df_list.append(df_stories)

    combined_df = pd.concat(df_list, ignore_index=True)
    
    combined_df = combined_df.rename(columns={
    'end_date': 'Snap.End Date',
    'start_date': 'Snap.Start Date',
    'aggregated_metrics.sold_impressions': 'Snap.Impressions',
    'aggregated_metrics.ecpm': 'Snap.ECPM',
    'aggregated_metrics.revenue': 'Snap.Revenue',
    'name': 'Snap.Name',
    'id': 'Snap.ID',
    'is_available': 'Snap.Is Available?'
})

combined_df['Snap.End Date'] = pd.to_datetime(combined_df['Snap.End Date'], errors='coerce').dt.tz_localize(None)
combined_df['Snap.Start Date'] = pd.to_datetime(combined_df['Snap.Start Date'], errors='coerce').dt.tz_localize(None)
unconverted_end_dates = combined_df[combined_df['Snap.End Date'].isna()]
unconverted_start_dates = combined_df[combined_df['Snap.Start Date'].isna()]

combined_df['_BATCH_LAST_RUN_'] = pd.Timestamp.now().date()

domo.write_dataframe(combined_df, 'Snapchat | Combined Revenue | Output', update_method="APPEND")

A newer version of domojupyter is available. Update domojupyter with the following command:

	conda install -c https://domo-conda-prod.s3.amazonaws.com/domo domojupyter=1.1.32
